In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import torch
from torchtext.data import Field, TabularDataset, Iterator, BucketIterator
from sklearn.model_selection import train_test_split
import random
import tqdm


In [2]:
def split_data(csv_file):
    data = pd.read_csv(csv_file)[['genre','lyrics']].dropna()
    x = data.drop('genre',axis=1)
    y = data.genre
    X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.5)
    train = X_train.join(y_train,how='outer')
    test = X_test.join(y_test,how ='outer')
    return train, test

In [3]:
train, test = split_data('data/lyrics.csv')


In [95]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('./model/GoogleNews-vectors-negative300.bin', binary=True)

FileNotFoundError: [Errno 2] No such file or directory: './model/GoogleNews-vectors-negative300.bin'

In [122]:
def tokenize(text):
    tokenized = []
    sentences = text.split('\n')
    for strings in sentences:
        tokenized.append(word_tokenize(strings))
    
    return tokenized

In [126]:
count = 0 

In [128]:
train
    

,lyrics,genre_Country,genre_Electronic,genre_Folk,genre_Hip-Hop,genre_Indie,genre_Jazz,genre_Metal,genre_Not Available,genre_Other,genre_Pop,genre_R&B,genre_Rock
0,OjalÃ¡ que llueva cafÃ© en el campo\nQue caiga...,0,0,0,0,0,0,0,0,0,0,0,1
1,"""Those who create literature know first-hand\n...",0,0,0,1,0,0,0,0,0,0,0,0
2,little creeper in the shade under my brain\nin...,0,0,0,0,0,0,0,0,0,0,0,1
3,This game is over\nI'm mean and older\nYou're ...,0,0,0,0,0,0,0,0,0,0,0,1
4,When I watched you walk with him\nTears filled...,0,0,0,0,0,0,0,0,0,0,0,1
5,"Ziggy, il sappelle ziggy\nCest mon seul ami\nD...",0,0,0,0,0,0,0,0,0,1,0,0
6,"By the sampling, the wealth of music and sound...",0,0,0,0,0,0,0,0,0,0,0,1
7,These are my famous last words\nMy number's up...,0,0,0,0,0,0,0,0,0,0,0,1
8,{instrumental},0,0,0,0,0,0,0,0,0,0,0,1
9,I don't know the circumstances\nAll I've heard...,0,0,0,0,0,0,1,0,0,0,0,0


In [129]:
train['lyrics']

0         OjalÃ¡ que llueva cafÃ© en el campo\nQue caiga...
1         "Those who create literature know first-hand\n...
2         little creeper in the shade under my brain\nin...
3         This game is over\nI'm mean and older\nYou're ...
4         When I watched you walk with him\nTears filled...
                                ...                        
133273    Ive slit the throats\nof clergymen and governo...
133274    It's up to you now\nThis place is filling up w...
133275    Pushed against a stone wall,\nYour heat betray...
133276    Gimme one good reason why I should listen to y...
133277    Talking (clearly making coke)\n"chorus" - -- I...
Name: lyrics, Length: 133278, dtype: object

In [130]:
def cleanup_data(dataframe):
    train = pd.concat([dataframe,pd.get_dummies(dataframe['genre'], prefix='genre')], axis=1)
    train.drop(['genre'], axis=1, inplace=True)
    train = train.reset_index()
    train = train.drop(['index'], axis=1)
    return train

In [131]:
train = cleanup_data(train)
test = cleanup_data(test)

KeyError: 'genre'

In [132]:
def output_train_test_csv(train,test):
    train.to_csv('data/train.csv', header=True)
    test.to_csv('data/test.csv', header=True)
    

In [133]:
output_train_test_csv(train,test)

In [134]:
import nltk, string

In [135]:
stop_words = nltk.corpus.stopwords.words('english') + ['\n'] + [i for i in string.punctuation]

In [171]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = English()
tokenizer = Tokenizer(nlp.vocab)


In [181]:
TEXT = Field(sequential=True, tokenize=tokenizer, fix_length=20,
             stop_words=stop_words)
LABEL = Field(sequential=False, use_vocab=True)


In [182]:
%%time
lyric_datafields = [("id", None), ("lyrics", TEXT), ("genre_Country", LABEL),("genre_Electronic", LABEL), ("genre_Folk", LABEL),("genre_Hip-Hop", LABEL),("genre_Indie", LABEL),
                    ("genre_Jazz", LABEL),("genre_Metal", LABEL),("genre_Not Available", LABEL),
                    ("genre_Other", LABEL),("genre_Pop", LABEL), ("genre_R&B", LABEL),  ("genre_Rock", LABEL)]

CPU times: user 11 µs, sys: 0 ns, total: 11 µs
Wall time: 16.7 µs


In [183]:
trn, vld = TabularDataset.splits(
        path="data", # the root directory where the data lies
        train='train.csv', validation="test.csv",
        format='csv',
        skip_header=False, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
        fields=lyric_datafields)

In [184]:
TEXT.build_vocab(trn)
LABEL.build_vocab(trn)


In [207]:
train_iter, val_iter = BucketIterator.splits(
        (trn, vld), # we pass in the datasets we want the iterator to draw data from
        batch_sizes=(100),
        device="cuda:01", # if you want to use the GPU, specify the GPU number here
        sort_key=lambda x: len(x.lyrics), # the BucketIterator needs to be told what function it should use to group the data.
        sort_within_batch=False,
        repeat=False # we pass repeat=False because we want to wrap this Iterator layer.
)

TypeError: 'int' object is not subscriptable

In [ ]:
TEXT.vocab.__dict__.keys()

In [ ]:
trn[10].lyrics

In [ ]:
batch = next(train_iter.__iter__()); batch

In [ ]:
batch.__dict__.keys()


In [ ]:
class BatchWrapper:
    def __init__(self, dl, x_var, y_vars):
        self.dl, self.x_var, self.y_vars = dl, x_var, y_vars # we pass in the list of attributes for x and y
    
    def __iter__(self):
        for batch in self.dl:
            x = getattr(batch, self.x_var) # we assume only one input in this wrapper
            
            if self.y_vars is not None: # we will concatenate y into a single tensor
                y = torch.cat([getattr(batch, feat).unsqueeze(1) for feat in self.y_vars], dim=1).float()
            else:
                y = torch.zeros((1))

            yield (x, y)
    
    def __len__(self):
        return len(self.dl)
    


In [ ]:
labels = []
for i in trn[0].__dict__.keys():
    labels.append(i)
labels = labels[1:]

In [ ]:

train_dl = BatchWrapper(train_iter, "lyrics", labels)
val_dl = BatchWrapper(val_iter, "lyrics", labels)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [ ]:
class SimpleBiLSTMBaseline(nn.Module):
    def __init__(self, hidden_dim, emb_dim=300,
                 spatial_dropout=0.05, recurrent_dropout=0.1, num_linear=2):
        super().__init__() # don't forget to call this!
        self.embedding = nn.Embedding(len(TEXT.vocab), emb_dim)
        self.encoder = nn.LSTM(emb_dim, hidden_dim, num_layers=1, dropout=recurrent_dropout)
        self.linear_layers = []
        for _ in range(num_linear - 1):
            self.linear_layers.append(nn.Linear(hidden_dim, hidden_dim))
        self.linear_layers = nn.ModuleList(self.linear_layers)
        self.predictor = nn.Linear(hidden_dim, 12)
    
    def forward(self, seq):
        hdn, _ = self.encoder(self.embedding(seq))
        feature = hdn[-1, :, :]
        for layer in self.linear_layers:
            feature = layer(feature)
        preds = self.predictor(feature)
        return preds

In [208]:
em_sz = 100
nh = 500
nl = 3
device = "cuda:01"
model = SimpleBiLSTMBaseline(nh, emb_dim=em_sz).to(device); model

RuntimeError: CUDA out of memory. Tried to allocate 12.94 GiB (GPU 1; 10.92 GiB total capacity; 1.27 GiB already allocated; 6.80 GiB free; 195.14 MiB cached)

In [ ]:
opt = optim.Adam(model.parameters(), lr=1e-2)
loss_func = nn.BCEWithLogitsLoss()

In [ ]:
epochs = 10



In [ ]:
%%time
for epoch in range(1, epochs + 1):
    running_loss = 0.0
    running_corrects = 0
    model.train() # turn on training mode
    for x, y in tqdm.tqdm(train_dl): # thanks to our wrapper, we can intuitively iterate over our data!
        opt.zero_grad()
        preds = model(x)
        loss = loss_func(preds, y)
        loss.backward()
        opt.step()

        running_loss += loss.data.item() * x[0].size()[0]
    epoch_loss = running_loss / len(trn)
    
    # calculate the validation loss for this epoch
    val_loss = 0.0
    model.eval() # turn on evaluation mode
    for x, y in val_dl:
        preds = model(x)
        loss = loss_func(preds, y)
        val_loss += loss.data.item() * x[0].size()[0]

    val_loss /= len(vld)
    print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}'.format(epoch, epoch_loss, val_loss))